In [65]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xg

In [66]:
# DE-SERIALIZE PICKLED NATURAL PRODUCT FINGERPRINTS
with open('/Users/marenaisabelle/CODE/GADSI/Capstone/np_prints') as fnpps:
    npps = pickle.load(fnpps)

In [67]:
# DE-SERIALIZE PICKLED XGB LOGISTIC CLASSIFIER
with open('/Users/marenaisabelle/CODE/GADSI/Capstone/OptimalXGB') as fmod:
    xgb_opt = pickle.load(fmod)

In [113]:
# LOAD NATURAL PRODUCT DF
np_df = pd.read_csv('/Users/marenaisabelle/Downloads/ibs2016oct_nc.smi', header = None)
np_df['stock_num'] = [x.split('\t')[1] for x in np_df[0]]
np_df['Smiles'] = [x.split('\t')[0] for x in np_df[0]]
np_df = np_df.drop(0, axis = 1)
print np_df.info()
np_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64576 entries, 0 to 64575
Data columns (total 2 columns):
stock_num    64576 non-null object
Smiles       64576 non-null object
dtypes: object(2)
memory usage: 1009.1+ KB
None


,stock_num,Smiles
0,STOCK1N-00001,O[C@H]1C=C2CC[N+]3([C@H]2[C@@H]([C@@H]1O)c1cc2...
1,STOCK1N-00002,COc1c2c(CC[C@@H](c3c2ccc(c(=O)c3)NCc2ccc(cc2)F...
2,STOCK1N-00004,COc1cc2CC3(CCCCC3)N=Cc2cc1OC.Cl
3,STOCK1N-00006,C=CCOc1ccc(cc1)C(=O)NC(C1COc2c(O1)cccc2)C
4,STOCK1N-00007,COc1cc2CN(C/C=C/c3ccccc3)CC3(c2cc1OC)CCOCC3.Cl


In [114]:
# PREDICT TEST COMPOUND ACTIVITIES
nppred = xgb_opt.predict(npps).tolist()
np_df['Pred_Activity'] = [x for x in nppred] # Append predicted activities to test df

In [115]:
# EVALUATE HIT CONFIDENCE/PROBABILITY
pred_proba = xgb_opt.predict_proba(npps)
np_df['Pred_Prob_Neg'] = [x[0] for x in pred_proba] # Append predicted probs to test df
np_df['Pred_Prob_Pos'] = [x[1] for x in pred_proba] # Append predicted probs to test df

In [117]:
# CLEAN AND VISUALIZE FINAL DATA FRAME
print np_df.describe()
np_df.head()

       Pred_Activity  Pred_Prob_Neg  Pred_Prob_Pos
count   64576.000000   64576.000000   64576.000000
mean        0.000449       0.979646       0.020354
std         0.021187       0.028801       0.028801
min         0.000000       0.109873       0.004436
25%         0.000000       0.978728       0.009788
50%         0.000000       0.986829       0.013171
75%         0.000000       0.990212       0.021272
max         1.000000       0.995564       0.890127


,stock_num,Smiles,Pred_Activity,Pred_Prob_Neg,Pred_Prob_Pos
0,STOCK1N-00001,O[C@H]1C=C2CC[N+]3([C@H]2[C@@H]([C@@H]1O)c1cc2...,0.0,0.975480,0.024520
1,STOCK1N-00002,COc1c2c(CC[C@@H](c3c2ccc(c(=O)c3)NCc2ccc(cc2)F...,0.0,0.990995,0.009005
2,STOCK1N-00004,COc1cc2CC3(CCCCC3)N=Cc2cc1OC.Cl,0.0,0.959863,0.040137
3,STOCK1N-00006,C=CCOc1ccc(cc1)C(=O)NC(C1COc2c(O1)cccc2)C,0.0,0.976744,0.023256
4,STOCK1N-00007,COc1cc2CN(C/C=C/c3ccccc3)CC3(c2cc1OC)CCOCC3.Cl,0.0,0.962025,0.037975


In [121]:
# RETURN INDICES OF ACTIVE COMPOUNDS
act_hits = np_df[np_df['Pred_Activity'] == 1].index.tolist()
print act_hits

[2795, 3444, 5162, 7455, 10818, 10852, 10853, 10917, 11062, 12072, 12719, 13098, 15061, 15067, 15116, 15575, 16985, 17173, 17691, 18628, 22632, 25783, 25858, 26002, 27003, 32920, 34022, 35603, 37303]


In [126]:
act_prob = [np_df.Pred_Prob_Pos.ix[x] for x in act_hits]
print act_prob

[0.59658271074295044, 0.52850592136383057, 0.57088911533355713, 0.68933278322219849, 0.53584879636764526, 0.54016953706741333, 0.62677860260009766, 0.56979376077651978, 0.63394749164581299, 0.59735476970672607, 0.56467336416244507, 0.50832110643386841, 0.64100879430770874, 0.71892303228378296, 0.54941999912261963, 0.83173567056655884, 0.50011807680130005, 0.73186153173446655, 0.52439314126968384, 0.57066303491592407, 0.62881487607955933, 0.51238405704498291, 0.51307779550552368, 0.51549327373504639, 0.52850592136383057, 0.51307779550552368, 0.86720371246337891, 0.87388944625854492, 0.89012742042541504]


In [132]:
print np.mean(act_prob)
print np.std(act_prob)

0.616306742717
0.117276465669


In [2]:
from sklearn.metrics import confusion_matrix